In [27]:
from datetime import date
from tqdm import tqdm
import shutil
import os

from thumbnails import ThumbnailDef, Thumbnailer, ThumbnailParams
from db_connect import DbConnector
from photo_export import PhotoExporter
from path_management import PathManager
from bucket_connect import BucketConnector

In [17]:
from_date = date(2011, 8, 1)
to_date = date(2011, 8, 25)

In [18]:
small = ThumbnailDef(120, 'small')
medium = ThumbnailDef(400, 'medium')
large = ThumbnailDef(1600, 'large')

sizes = [small, large, medium] 
thumbnailer = Thumbnailer(sizes)

In [19]:
paths = PathManager(thumbnailer.name_modifiers)
test_connector = DbConnector('./test.postgres_config', paths)
prod_connector = DbConnector('./prod.postgres_config', paths)
exporter = PhotoExporter(paths)

games = prod_connector.get_games(from_date, to_date)

In [21]:
for game in tqdm(games):
    exporter.export_photos_for_game(game)

 29%|██▊       | 2/7 [00:00<00:01,  3.57it/s]

skipping missing photo 74D652BA-22F9-4ECC-89EC-2CF32C060EAB.JPG
skipping missing photo 94D8E58D-62E3-4565-8ADE-BF7AC13BED48.JPG
skipping missing photo A81B0891-97BA-4ADA-B347-EA9E69FAC50E.JPG


100%|██████████| 7/7 [00:01<00:00,  5.26it/s]

skipping missing photo A843C736-6C6A-4793-ADD7-43F40AACA258.JPG
skipping missing photo A390A4B8-E325-437F-8C7A-9DD39EC5E257.JPG


At this point, review the photos in the "preview" directories and remove any that shouldn't be uploaded. Removing it from the preview directory is sufficient.

In [22]:
to_thumbnail: list[ThumbnailParams] = []
for game in games:
    to_thumbnail.extend(exporter.get_photos_to_thumbnail(game))

In [23]:
thumbnailer.thumbnail_many(to_thumbnail)

In [24]:
bucket = BucketConnector('./prod.bucket_config', paths)

In [25]:
for game in games:
    to_upload = exporter.get_photos_to_upload(game)
    print(f'Uploading/Validating {len(to_upload)} photos for {game.Name}')
    for photo in to_upload:
        root_path = paths.temp_dir(game, photo)
        bucket.upload_all_files(root_path, photo)
    prod_connector.import_resources(game, to_upload)

Uploading/Validating 0 photos for 8/23/11 Atlanta Braves at Chicago Cubs
Uploading/Validating 16 photos for 8/21/11 St. Louis Cardinals at Chicago Cubs
Uploading/Validating 18 photos for 8/20/11 St. Louis Cardinals at Chicago Cubs
Uploading/Validating 0 photos for 8/16/11 Los Angeles Dodgers at Milwaukee Brewers
Uploading/Validating 0 photos for 8/11/11 Washington Nationals at Chicago Cubs
Uploading/Validating 0 photos for 8/8/11 Washington Nationals at Chicago Cubs
Uploading/Validating 0 photos for 8/7/11 Cincinnati Reds at Chicago Cubs


In [28]:
for game in games:
    root_path = paths.base_dir(game)
    if os.path.exists(root_path):
        shutil.rmtree(root_path)